In [1]:
import numpy as np
from Meshing.Meshing import *
from Fields.Fields import *
from Kernels.Kernels import *
from Solvers.Solvers import *

In [11]:
# initial parameters / user input
nZones = 5
areas = [1]*nZones
L0 = 0
L1 = 1
coords = np.linspace(L0,L1,nZones)

# time delta t
_dt = 1e321

# initial values
pressure_initial = 0.0
mdot_initial = 1.0
h_initial = 1.0
rho_initial = 1.0

# BCs
mdot_bc = 1.0
pressure_bc = 0.0
h_bc = 1.0
rho_bc = 1.0


# Make a mesh
mesh = Mesh_1D(nodeCoords=coords, faceAreas=areas)

# Make fields for each variable
pressure = ScalarField(name='P', initial_value=pressure_initial, mesh=mesh)
mdot = ScalarField(name='mdot', initial_value=mdot_initial, mesh=mesh)
h = ScalarField(name='enthalpy', initial_value=h_initial, mesh=mesh)
rho = ScalarField(name='density', initial_value=rho_initial, mesh=mesh)

A_mass = np.zeros([mesh.nz, mesh.nz])
A_energy = np.zeros([mesh.nz, mesh.nz])
A_momentum = np.zeros([mesh.nz, mesh.nz])

b_mass = np.zeros([mesh.nz])
b_energy = np.zeros([mesh.nz])
b_momentum = np.zeros([mesh.nz])

# Now iterate and add coeffs to each matrix.
for cid in mesh.cidList:
  area = ( mesh.cells[cid].upperArea + mesh.cells[cid].lowerArea ) / 2.0
  dz = mesh.cells[cid].dz
  A_X_dt = area*dz / _dt
  A_dt = area / _dt
  X_dt = dz / _dt

  # Solve implicitely for mass flow rates based on densities
  ### A_X_dt * (rho - rho_old) + (m - m_i-1) = 0.0
  b_mass[cid] =  -1.0 * A_X_dt * (rho.T[cid] - rho.T_old[cid])
  A_mass[cid,cid] = 1.0
  if cid == 0: # Boundary
    b_mass[cid] += mdot_bc
  else:
    A_mass[cid,cid-1] = -1.0

np.linalg.solve(A_mass, b_mass)




array([1., 1., 1., 1.])

In [12]:
A_mass

array([[ 1.,  0.,  0.,  0.],
       [-1.,  1.,  0.,  0.],
       [ 0., -1.,  1.,  0.],
       [ 0.,  0., -1.,  1.]])